### Creating sqlite database and exporting csv data into the created database
#### <li>Load csv data</li>
#### <li>Create/Connect to sqlite database</li>
#### <li>Load data file to sqlite</li>
#### <li>Close connection</li>

In [1]:
import sqlite3
import pandas as pd

#Step 1: Load csv data
data=pd.read_csv('buildings.csv')

#Step 2: Create/Connect to sqlite database
conn=sqlite3.connect('PFSIN.db')

#step 3: Load data file to sqlite
data.to_sql('properties',conn,if_exists='replace')

#step 4: Close Connection
conn.close()

In [ ]:
#Load and activate the SQL extension
%load_ext sql

In [ ]:
%sql sqlite:///PFSIN.db

# Database Normalization
### Original table is in 1NF with the following columns
<img src='Original_table.png'>

### Create additional tables to implement normalization

In [23]:
%%sql
DROP TABLE IF EXISTS amenities;
CREATE TABLE amenities(
    amenity_id INTEGER PRIMARY KEY AUTOINCREMENT,
    house_id INTEGER,
    has_borehole INTEGER,
    has_garden INTEGER,
    has_parking_bay INTEGER,
    has_generator INTEGER,
    has_balcony INTEGER,
    has_cctv INTEGER,
    has_fibre_internet INTEGER,
    has_gym INTEGER,
    FOREIGN KEY (house_id) REFERENCES properties(house_id)
);

DROP TABLE IF EXISTS regions;
CREATE TABLE regions(
    region_id INTEGER PRIMARY KEY AUTOINCREMENT,
    region_name TEXT
);
DROP TABLE IF EXISTS bedrooms;
CREATE TABLE bedrooms(
    bedroom_id INTEGER PRIMARY KEY AUTOINCREMENT,
    no_bedrooms INTEGER
);
DROP TABLE IF EXISTS bathrooms;
CREATE TABLE bathrooms(
    bathroom_id INTEGER PRIMARY KEY AUTOINCREMENT,
    no_bathrooms INTEGER
);
DROP TABLE IF EXISTS location;
CREATE TABLE location(
    location_id INTEGER PRIMARY KEY AUTOINCREMENT,
    house_id INTEGER,
    longitude REAL,
    latitude REAL,
    FOREIGN KEY (house_id) REFERENCES properties(house_id)
);
DROP TABLE IF EXISTS prices;
CREATE TABLE prices(
    price_id INTEGER PRIMARY KEY AUTOINCREMENT,
    house_id INTEGER,
    price INTEGER,
    FOREIGN KEY (house_id) REFERENCES properties(house_id)
);
DROP TABLE IF EXISTS buildings;
CREATE TABLE buildings (
    house_id INTEGER PRIMARY KEY AUTOINCREMENT,
    building_name TEXT,
    bedroom_id INTEGER,
    region_id INTEGER,
    bathroom_id INTEGER,
    FOREIGN KEY (bathroom_id) REFERENCES bathrooms(bathroom_id),
    FOREIGN KEY (region_id) REFERENCES regions(region_id),
    FOREIGN KEY (bedroom_id) REFERENCES bedrooms(bedroom_id)
);

 * sqlite:///PFSIN.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

### Add data to the created tables

In [26]:
%%sql
INSERT INTO amenities (house_id,has_borehole,has_garden,has_parking_bay,has_generator,has_balcony,has_cctv,
                       has_fibre_internet,has_gym)
SELECT
    house_id,
    has_borehole,
    has_garden,
    has_parking_bay,
    has_generator,
    has_balcony,
    has_cctv,
    has_fibre_internet,
    has_gym
FROM properties;

INSERT INTO regions(region_name)
SELECT
    DISTINCT region
FROM properties
WHERE region IS NOT NULL;

INSERT INTO bedrooms(no_bedrooms)
SELECT
    DISTINCT number_of_bedrooms
FROM properties
WHERE number_of_bedrooms IS NOT NULL;

INSERT INTO bathrooms(no_bathrooms)
SELECT
    DISTINCT number_of_bathrooms
FROM properties
WHERE number_of_bathrooms IS NOT NULL;

INSERT INTO prices(house_id,price)
SELECT
    house_id,
    price
FROM properties;

INSERT INTO location(house_id,longitude,latitude)
SELECT
    house_id,
    longitude,
    latitude
FROM properties;

WITH properties_1 AS(
    SELECT
        properties.house_id,
        properties.building_name,
        regions.region_id,
        bedrooms.bedroom_id,
        bathrooms.bathroom_id
    FROM properties
    LEFT JOIN
        bedrooms
        ON properties.number_of_bedrooms = bedrooms.no_bedrooms
    LEFT JOIN
        bathrooms
        ON properties.number_of_bathrooms = bathrooms.no_bathrooms
    LEFT JOIN
        regions
        ON properties.region = regions.region_name
)
INSERT INTO buildings (house_id,building_name,bedroom_id,region_id,bathroom_id)
SELECT
    house_id,
    building_name,
    bedroom_id,
    region_id,
    bathroom_id
FROM properties_1;

DROP TABLE properties;
ALTER TABLE buildings RENAME TO properties;

 * sqlite:///PFSIN.db
9 rows affected.
10 rows affected.
1133 rows affected.
1133 rows affected.


[]

## Resulting ER Diagram after Normalization
<img src='ER_diagram.png'><img>